<a href="https://colab.research.google.com/github/arpitarumma/DataPrivacySecurity/blob/main/enhancedCiphergate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bcrypt gradio
!pip install gradio



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.7 MB/s eta 0:00:00


In [2]:
import sqlite3

def initialize_database():
    """Creates a users table if it does not exist."""
    conn = sqlite3.connect("users.db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL
    )
    """)

    conn.commit()
    conn.close()
    print("✅ Database initialized successfully!")

# Initialize the database
initialize_database()


✅ Database initialized successfully!


In [4]:
import sqlite3
import bcrypt

# Create a database and users table if not exists
conn = sqlite3.connect("users.db")
cursor = conn.cursor()

# Changed 'password' column to 'password_hash' for consistency with initialize_database()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password_hash TEXT
    )
""")
conn.commit()

# Function to register users
def register():
    username = input("Enter a new username: ")
    password = input("Enter a new password: ")

    # Hash the password
    hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())

    try:
        # Changed 'password' to 'password_hash' to match the table schema
        cursor.execute("INSERT INTO users (username, password_hash) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        print("✅ Registration successful! You can now log in.")
    except sqlite3.IntegrityError:
        print("❌ Username already exists! Try again.")

# Function to login users
def login():
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    # Changed 'password' to 'password_hash' to match the table schema
    cursor.execute("SELECT password_hash FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()

    if user:
        stored_password = user[0]
        if bcrypt.checkpw(password.encode(), stored_password):
            print("✅ Login successful! Welcome,", username)
        else:
            print("❌ Invalid username or password.")
    else:
        print("❌ Invalid username or password.")

# Main menu
while True:
    choice = input("\n1. Register\n2. Login\n3. Exit\nChoose an option: ")
    if choice == "1":
        register()
    elif choice == "2":
        login()
    elif choice == "3":
        print("Exiting...")
        break
    else:
        print("❌ Invalid choice! Please select 1, 2, or 3.")


1. Register
2. Login
3. Exit
Choose an option: 1
Enter a new username: arpita
Enter a new password: 1234arpita
✅ Registration successful! You can now log in.

1. Register
2. Login
3. Exit
Choose an option: 2
Enter your username: arpita
Enter your password: 1234arpita
✅ Login successful! Welcome, arpita

1. Register
2. Login
3. Exit
Choose an option: 3
Exiting...


In [10]:
import sqlite3

conn = sqlite3.connect("users.db")
cursor = conn.cursor()

# Delete old table (⚠ WARNING: This will remove all users)
cursor.execute("DROP TABLE IF EXISTS users")

# Create correct table structure
cursor.execute("""
    CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL
    )
""")

conn.commit()
conn.close()

print("✅ Table recreated successfully!")


✅ Table recreated successfully!


In [11]:
import sqlite3, bcrypt

conn = sqlite3.connect("users.db")
cursor = conn.cursor()

username = "testuser"
password = "test123"
hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt())

cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_pw.decode()))
conn.commit()
conn.close()

print("✅ Test user registered successfully!")


✅ Test user registered successfully!


In [12]:
import gradio as gr
import sqlite3
import bcrypt

# Database Setup
DB_NAME = "users.db"

def create_table():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL
    )
    """)
    conn.commit()
    conn.close()

create_table()  # Ensure table exists

# Function to register users
def register(username, password):
    if not username or not password:
        return "⚠️ Username and password cannot be empty!"

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Check if username already exists
    cursor.execute("SELECT * FROM users WHERE username=?", (username,))
    if cursor.fetchone():
        conn.close()
        return "❌ Username already taken!"

    # Hash password before storing
    hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt())

    cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_pw))
    conn.commit()
    conn.close()

    return "✅ Registration successful! Please login."

# Function to verify login
def login(username, password):
    if not username or not password:
        return "⚠️ Enter both username and password!"

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("SELECT password FROM users WHERE username=?", (username,))
    result = cursor.fetchone()

    conn.close()

    if result and bcrypt.checkpw(password.encode(), result[0]):
        return f"🎉 Welcome, {username}!"
    else:
        return "❌ Invalid username or password."

# Gradio UI
with gr.Blocks(theme=gr.themes.Base()) as demo:
    gr.Markdown("<h1 style='text-align: center; color: gold;'>🔐 CipherGate: Secure Login System</h1>")

    with gr.Tabs():
        with gr.Tab("🔑 Register"):
            with gr.Row():
                username_r = gr.Textbox(label="Username")
                password_r = gr.Textbox(label="Password", type="password")
            register_btn = gr.Button("📝 Register", variant="primary")
            output_r = gr.Textbox(label="Output", interactive=False)
            register_btn.click(register, inputs=[username_r, password_r], outputs=output_r)

        with gr.Tab("🔒 Login"):
            with gr.Row():
                username_l = gr.Textbox(label="Username")
                password_l = gr.Textbox(label="Password", type="password")
            login_btn = gr.Button("✅ Login", variant="primary")
            output_l = gr.Textbox(label="Output", interactive=False)
            login_btn.click(login, inputs=[username_l, password_l], outputs=output_l)

# Run the Gradio app
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b8eaaeab54e798747.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
import sqlite3
conn = sqlite3.connect("users.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())  # Should show [('users',)]
conn.close()


[('sqlite_sequence',), ('users',)]


In [14]:
import sqlite3

conn = sqlite3.connect("users.db")
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(users)")
columns = cursor.fetchall()

for col in columns:
    print(col)

conn.close()



(0, 'id', 'INTEGER', 0, None, 1)
(1, 'username', 'TEXT', 1, None, 0)
(2, 'password', 'TEXT', 1, None, 0)
